In [53]:
from dotenv import load_dotenv
load_dotenv()
from google.cloud.firestore import Client as FirestoreClient
from chat_history_service import ChatHistoryService
from llm_request_service import LLMRequestService
from typing import List, Dict, Any
import pandas as pd
from textwrap import fill

## Define Functions

In [54]:
# def get_chat_history_from_firestore(session_id: str) -> list[dict]:
#     """
#     Fetch chat history from Firestore for a given session ID.
    
#     Args:
#         session_id (str): The session ID to fetch chat history for.
    
#     Returns:
#         list[dict]: List of chat messages in the session.
#     """
#     chat_history_service = ChatHistoryService()
#     chat_history = chat_history_service.get_chat_history_by_id(session_id)
#     chat_history = sorted(chat_history, key=lambda x: x.get("create_time", ""))
#     print(chat_history)
#     return chat_history

In [55]:
def get_chat_history_from_firestore(session_ids: list[str]) -> dict[str, list[dict]]:
    """
    Fetch chat histories from Firestore for given session IDs.
    
    Args:
        session_ids (list[str]): The list of session IDs to fetch chat histories for.
    
    Returns:
        dict[str, list[dict]]: Dictionary of session IDs and their respective chat histories.
    """
    chat_history_service = ChatHistoryService()
    chat_histories = chat_history_service.get_chat_histories_by_ids(session_ids)
    
    for session_id, history in chat_histories.items():
        chat_histories[session_id] = sorted(history, key=lambda x: x.get("create_time", ""))
    
    print(chat_histories)
    return chat_histories

In [56]:
def get_specific_data_from_history(
    history: List[Dict[str, Any]],
) -> List[Dict[str, Any]]:
    """
    Retrieves specific data from the entire conversation history, maintaining the conversation flow.

    Args:
        history (List[Dict[str, Any]]): List of dictionaries containing message history.
    s
    Returns:
        List[Dict[str, Any]]: List of dictionaries, each containing user message, bot responses, and button label for each turn in the conversation.
    """
    # Sort the history by create_time
    sorted_history = sorted(history, key=lambda x: x.get("create_time"))

    # Initialize list to store the conversation turns
    conversation = []

    # Iterate through the history
    for message in sorted_history:
        turn = {}

        # Preserve the order of events and skip None values
        if "user_msg" in message and message["user_msg"] is not None:
            turn["user_msg"] = message["user_msg"]
        if "button_label" in message and message["button_label"] is not None:
            turn["button_label"] = message["button_label"]
        if "bot_response" in message and message["bot_response"] is not None:
            bot_response = message["bot_response"]
            if isinstance(bot_response, list):
                bot_messages = []
                for response in bot_response:
                    if isinstance(response, dict) and "message" in response and response["message"] is not None:
                        bot_messages.append({"message": response["message"]})
                if bot_messages:
                    turn["bot_response"] = bot_messages

        # Add the turn to the conversation if it's not empty
        if turn:
            conversation.append(turn)

    return conversation

In [57]:
def get_specific_chat_data(chat_history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Selects specific data from the chat history.
    Args:
        chat_history (List[Dict[str, Any]]): The chat history to process.
    Returns:
        List[Dict[str, Any]]: Selected specific data from the chat history.
    """
    specific_chat_history = get_specific_data_from_history(chat_history)
    return specific_chat_history

In [58]:
def get_specific_chat_data_for_multiple_sessions(
    chat_histories: Dict[str, List[Dict[str, Any]]]
) -> Dict[str, List[Dict[str, Any]]]:
    """
    Fetches and processes chat histories for multiple sessions.
    Args:
        chat_histories (Dict[str, List[Dict[str, Any]]]): Dictionary of session IDs and their corresponding chat histories.
    Returns:
        Dict[str, List[Dict[str, Any]]]: Dictionary of session IDs and their processed specific chat data.
    """
    processed_data = {}
    for session_id, history in chat_histories.items():
        processed_data[session_id] = get_specific_data_from_history(history)
    return processed_data

In [59]:
def create_pretty_dataframe(processed_data):
    rows = []
    for session_id, conversation in processed_data.items():
        dialogue = ""
        for turn in conversation:
            # Check for button label first
            if 'button_label' in turn and turn['button_label'] is not None:
                dialogue += f"User button: {turn['button_label']}\n\n"
            
            # Then check for user message
            if 'user_msg' in turn and turn['user_msg'] is not None:
                dialogue += f"User: {turn['user_msg']}\n\n"
            
            # Finally, add bot responses
            if 'bot_response' in turn and turn['bot_response'] is not None:
                for response in turn['bot_response']:
                    if isinstance(response, dict) and 'message' in response and response['message'] is not None:
                        dialogue += f"Bot: {response['message']}\n"
            
            dialogue += "\n"

        rows.append({'Session ID': session_id, 'Dialogue': dialogue.strip()})
    
    dialogue_df = pd.DataFrame(rows)
    return dialogue_df

In [60]:
# def call_llm_service_summarize(conversation: str, prompt: str = "") -> str:
#     """
#     Call the LLM service to summarize the chat history.
#     Args:
#         conversation (str): The chat history to summarize.
#         prompt (str): The prompt to send to the LLM service.
#             If not provided, the default prompt will be fetched from Firestore.
#     Returns:
#         str: The summary of the chat history.
#     """
#     llm_request_service = LLMRequestService()
#     chat_history_summary = llm_request_service.summarize_conversation(
#         conversation, prompt
#     )
#     return chat_history_summary

In [61]:
def call_llm_service_summarize(conversations: Dict[str, str], prompt: str = "") -> Dict[str, str]:
    """
    Call the LLM service to summarize multiple chat histories.
    Args:
        conversations (Dict[str, str]): A dictionary of chat histories to summarize, 
                                        where the key is the session ID and the value is the conversation.
        prompt (str): The prompt to send to the LLM service.
                      If not provided, the default prompt will be fetched from Firestore.
    Returns:
        Dict[str, str]: A dictionary of summaries, where the key is the session ID and the value is the summary.
    """
    llm_request_service = LLMRequestService()
    chat_history_summaries = llm_request_service.summarize_conversations(
        conversations, prompt
    )
    return chat_history_summaries

In [62]:
def format_summary(summary: str) -> str:
    # Split the summary into sentences
    sentences = summary.split(". ")
    # Join the sentences with line breaks
    formatted_summary = ".\n".join(sentences)
    return f"Summary:\n{formatted_summary}"


def count_output(text):
    """
    Count characters, words, and lines in the given text.
    """
    char_count = len(text)
    word_count = len(text.split())
    return char_count, word_count

In [63]:
def call_llm_service_queue(
    summary: str,
    customer_comment: str = "",
    service: str = "",
    telesales: str = "",
    techniek: str = "",
    activatie: str = "",
    tag_prompt: str = "",
) -> str:
    """
    Call the LLM service to tag the call queue.
    Args:
        summary (str): The ai generated summary of the conversation.
        tag_prompt (str): The prompt to send to the LLM service for tagging.
    Returns:
        str: Tagged call queue.
    """
    llm_request_service = LLMRequestService()
    tag_queue = llm_request_service.tag_callback(
        summary, customer_comment, service, telesales, techniek, activatie, tag_prompt
    )
    return tag_queue

## Gather Conversation

### Get conversation from session_id in Development

In [64]:
# session_id = "00001cac-a1fe-493d-8c04-813527671906" # Replace with other session ID
# chat_history = get_chat_histories_from_firestore(session_id)
# chat_history

# conversation = get_specific_chat_data(chat_history)
# conversation

In [65]:
session_ids = [
    "00001a3e-6022-4d94-a506-143856c60b0e",
    "00000bdf-17cf-429e-aba9-bf5cfe4f9ba6",
    "00001339-f528-42df-b90f-d07ec400c96d"
]  
chat_histories = get_chat_history_from_firestore(session_ids)
chat_histories

{'00001a3e-6022-4d94-a506-143856c60b0e': [{'user_msg': None, 'response_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 460779, tzinfo=datetime.timezone.utc), 'bot_response': [{'message_en': "Hi, I'm chatbot Izzi from Odido.", 'type': 'text', 'message': 'Hallo, ik ben chatbot Izzi van Odido.'}, {'message_en': 'Which product is your question about?', 'language': 'nl', 'type': 'button-list', 'buttons': [{'label_en': 'Mobile', 'event': 'MT_SYSTEM_PRODUCTSELECT-choice;mobiel', 'label': 'Mobiel'}, {'label_en': 'Internet + TV', 'event': 'MT_SYSTEM_PRODUCTSELECT-choice;thuis', 'label': 'Internet + TV'}], 'escapeOption': False, 'message': 'Over welk product wil je iets vragen?'}], 'button_label': None, 'intent_id': 'MT_SYSTEM_PRODUCTSELECT', 'sender': 'bot', 'create_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 483898, tzinfo=datetime.timezone.utc)}, {'user_msg': None, 'response_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 30, 465481, tzinfo=datetime.timezone.utc), 'bot_respo

{'00001a3e-6022-4d94-a506-143856c60b0e': [{'user_msg': None,
   'response_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 460779, tzinfo=datetime.timezone.utc),
   'bot_response': [{'message_en': "Hi, I'm chatbot Izzi from Odido.",
     'type': 'text',
     'message': 'Hallo, ik ben chatbot Izzi van Odido.'},
    {'message_en': 'Which product is your question about?',
     'language': 'nl',
     'type': 'button-list',
     'buttons': [{'label_en': 'Mobile',
       'event': 'MT_SYSTEM_PRODUCTSELECT-choice;mobiel',
       'label': 'Mobiel'},
      {'label_en': 'Internet + TV',
       'event': 'MT_SYSTEM_PRODUCTSELECT-choice;thuis',
       'label': 'Internet + TV'}],
     'escapeOption': False,
     'message': 'Over welk product wil je iets vragen?'}],
   'button_label': None,
   'intent_id': 'MT_SYSTEM_PRODUCTSELECT',
   'sender': 'bot',
   'create_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 483898, tzinfo=datetime.timezone.utc)},
  {'user_msg': None,
   'response_time': 

In [66]:
# Process the histories to get specific data for multiple sessions
processed_data = get_specific_chat_data_for_multiple_sessions(chat_histories)

processed_data

{'00001a3e-6022-4d94-a506-143856c60b0e': [{'bot_response': [{'message': 'Hallo, ik ben chatbot Izzi van Odido.'},
    {'message': 'Over welk product wil je iets vragen?'}]},
  {'button_label': 'Internet + TV',
   'bot_response': [{'message': 'Komen we er samen niet uit, dan zijn onze adviseurs er voor je.'},
    {'message': 'Wat kan ik voor je doen?'}]},
  {'button_label': 'Bestelstatus',
   'bot_response': [{'message': 'Ik vertel je graag meer over de status van je bestelling.'},
    {'message': 'Heb je een mail ontvangen met de bevestiging van je bestelling?'}]},
  {'button_label': 'Andere vraag',
   'bot_response': [{'message': 'Een andere vraag kan natuurlijk ook. Hoe kan ik je helpen?'}]},
  {'user_msg': 'Adviseur spreken',
   'bot_response': [{'message': 'Ik ga voor je kijken of een adviseur je kan helpen.'},
    {'message': "Kies 'Verkoop' voor hulp bij bestellen of verlengen van een abonnement en/of telefoon. Kies 'Service' voor andere vragen."}]},
  {'button_label': 'Verkoop',

In [67]:
# Process the histories to get specific data for multiple sessions
processed_data = get_specific_chat_data_for_multiple_sessions(chat_histories)

# Create the pretty DataFrame
pretty_df = create_pretty_dataframe(processed_data)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)  # Adjust this value as needed

# Function to wrap text
def wrap_text(text, width=100):
    return '\n'.join(fill(line, width) for line in text.split('\n'))

# Apply text wrapping to the Dialogue column
pretty_df['Dialogue'] = pretty_df['Dialogue'].apply(wrap_text)

# Display the DataFrame
for index, row in pretty_df.iterrows():
    print(f"Session ID: {row['Session ID']}")
    print(row['Dialogue'])
    print("-" * 100)  # Separator
    print()

Session ID: 00001a3e-6022-4d94-a506-143856c60b0e
Bot: Hallo, ik ben chatbot Izzi van Odido.
Bot: Over welk product wil je iets vragen?

User button: Internet + TV

Bot: Komen we er samen niet uit, dan zijn onze adviseurs er voor je.
Bot: Wat kan ik voor je doen?

User button: Bestelstatus

Bot: Ik vertel je graag meer over de status van je bestelling.
Bot: Heb je een mail ontvangen met de bevestiging van je bestelling?

User button: Andere vraag

Bot: Een andere vraag kan natuurlijk ook. Hoe kan ik je helpen?

User: Adviseur spreken

Bot: Ik ga voor je kijken of een adviseur je kan helpen.
Bot: Kies 'Verkoop' voor hulp bij bestellen of verlengen van een abonnement en/of telefoon. Kies
'Service' voor andere vragen.

User button: Verkoop

Bot: Ik kan je op dit moment helaas niet doorverbinden met een adviseur.
Bot: Je kan onze adviseurs bellen op 0800-0092. Op maandag t/m vrijdag zijn we bereikbaar van 8-20
uur, op zaterdag van 9-17.30 uur, op zondag van 12-17.30 uur, op feestdagen zijn 

## Manage Summary Prompt

In [68]:
prompt = """
Vat deze chat {text} samen en beschrijf wat de (potentiële) klant van het telecombedrijf Odido wil. 
Focus alleen op de klantinput in "user_msg" en "button_label". Negeer begroetingen, afscheid, smalltalk en beledigingen. 
Benoem ook nooit dat de klant een terugbelverzoek wilt. 
Maak een bondige samenvatting in het Nederlands van maximaal 100 tekens. Begin met "Je...".
"""

## Generate Summaries

In [69]:
# summary = call_llm_service_summarize(conversations, prompt)
# formatted_output = format_summary(summary)
# print(formatted_output)
# print()

# # Count the output
# char_count, word_count = count_output(formatted_output)
# print(f"Characters: {char_count}")
# print(f"Words: {word_count}")

In [70]:
# Assuming you have chat_histories
conversations = get_specific_chat_data_for_multiple_sessions(chat_histories)

# Call the summarization function
summaries = call_llm_service_summarize(conversations, prompt)

# Print summaries
for session_id, summary in summaries.items():
    formatted_summary = format_summary(summary)
    print(f"Session ID: {session_id}")
    print(formatted_summary)
    
    char_count, word_count = count_output(formatted_summary)
    print(f"Characters: {char_count}")
    print(f"Words: {word_count}")
    print()

Session ID: 00001a3e-6022-4d94-a506-143856c60b0e
Summary:
Je wilt een adviseur spreken voor hulp bij bestellen of servicevragen.
Characters: 79
Words: 12

Session ID: 00000bdf-17cf-429e-aba9-bf5cfe4f9ba6
Summary:
Je hebt geen wifi meer en wilt hulp bij het oplossen van internetproblemen.
Characters: 84
Words: 14

Session ID: 00001339-f528-42df-b90f-d07ec400c96d
Summary:
Je wilt data aanvullen in zone 2, maar kunt dat niet doen met MB's.
Characters: 76
Words: 15



## Update the prompt in Firestore Database (code uses the uploaded one)

In [71]:
# firestore_client = FirestoreClient()
# collection = firestore_client.collection("llm-prompts")
# document = collection.document("fulfillment-webhook")
# document.update({"summarize-chat-conversation": prompt})

# Manage Parameters and Call Tag Queue Prompt

In [72]:
service = """
Voorbeelden: 
- Vragen over facturen en betalingen (bijvoorbeeld uitleg over de factuur, incassodatum, betalingstermijn)
- Verzoeken voor afbetalingsregelingen
- Vragen over extra kosten of inflatiecorrectie
- Assistentie bij pin/puk-codes
- Hulp met de Mijn Odido omgeving (inclusief wachtwoordproblemen)
- Wijzigen van persoonlijke of betaalgegevens
- Verzoeken voor contractsovername
- Aanvragen van een vervangende simkaart
- Assistentie bij nummerportering
- Informatie over dienstgebruik en tarieven
- Aanpassen van extra's of bundels
"""

telesales = """
Voorbeelden:
- Afsluiten van een nieuw abonnement
- Verlengen van een bestaand abonnement
- Opzeggen van een abonnement (zowel tijdens als na de contractperiode)
"""

techniek = """
Voorbeelden:
- Problemen met thuisinternet (geen verbinding of slechte kwaliteit)
- Problemen met mobiel internet
- Slechte TV-ontvangst of niet-werkende TV-kastjes
- Router-gerelateerde problemen
- Problemen met telefonisch bereik
- Niet-beschikbare TV-zenders
- Algemene storingen met diensten
"""

activatie = """
Voorbeelden:
- Vragen over de planning van de monteur
- Informatie over de aansluitdatum
- Problemen na aansluiting (bijvoorbeeld wel aangesloten maar geen diensten)
- Melden van niet-verschenen monteurs
"""


In [73]:
customer_comment = """er is een storing"""

In [74]:
tag_prompt = """
Analyseer de contactreden van een (potentiële) klant van Odido, een telecombedrijf, en categoriseer deze in de juiste klantenservice queue.

Gegeven informatie:
- Primaire contactreden: {text}
- Aanvullende klantopmerking: {customer_comment}
Als de aanvullende klantopmerking afwijkt van de primaire contactreden, hecht dan meer waarde aan de klantopmerking mits dit een inhoudelijke toevoeging is.

Beschikbare queues:
1. Queue: Service
Omschrijving: Behandelt vragen over facturering, administratieve zaken en algemene informatie over diensten en accounts.
Voorbeelden: {service}
2. Queue: Telesales
Omschrijving: Handelt abonnementsaanvragen, nieuwe bestellingen, verlengingen en opzeggingen af.
Voorbeelden: {telesales}
3. Queue: Techniek
Omschrijving: Lost technische problemen op gerelateerd aan internet, TV en mobiele diensten.
Voorbeelden: {techniek}
4. Queue: Activatie
Omschrijving: Begeleidt nieuwe klanten bij het aansluitproces van thuisinternet en/of TV.
Voorbeelden: {activatie}

Instructies:
1. Analyseer de contactreden en eventuele aanvullende opmerking.
2. Analyseer de queues met de bijbehorende omschrijvingen en voorbeelden.
3. Geef een beknopte argumentatie (maximaal 100 tekens) waarom de klantvraag bij een bepaalde queue zou passen.
4. Kies op basis van deze argumentatie de meest geschikte queue.

Vereiste output:
Genereer een JSON-object met de volgende structuur:
{{
    "queue": "<naam van de gekozen queue>",
    "reason": "<beknopte reden voor de gekozen queue>"
}}

Zorg ervoor dat de output strikt voldoet aan dit JSON-formaat, zonder extra witruimte of aanvullende tekst.
"""

# Tag Call Queue

In [75]:
call_queue = call_llm_service_queue(
    summary, customer_comment, service, telesales, techniek, activatie, tag_prompt
)
print(f"Tag prompt output: {call_queue}")

Summary: Je wilt data aanvullen in zone 2, maar kunt dat niet doen met MB's.
Customer Comment: er is een storing
Data prepared: {'custom': {'text': "Je wilt data aanvullen in zone 2, maar kunt dat niet doen met MB's.", 'customer_comment': 'er is een storing', 'service': "\nVoorbeelden: \n- Vragen over facturen en betalingen (bijvoorbeeld uitleg over de factuur, incassodatum, betalingstermijn)\n- Verzoeken voor afbetalingsregelingen\n- Vragen over extra kosten of inflatiecorrectie\n- Assistentie bij pin/puk-codes\n- Hulp met de Mijn Odido omgeving (inclusief wachtwoordproblemen)\n- Wijzigen van persoonlijke of betaalgegevens\n- Verzoeken voor contractsovername\n- Aanvragen van een vervangende simkaart\n- Assistentie bij nummerportering\n- Informatie over dienstgebruik en tarieven\n- Aanpassen van extra's of bundels\n", 'telesales': '\nVoorbeelden:\n- Afsluiten van een nieuw abonnement\n- Verlengen van een bestaand abonnement\n- Opzeggen van een abonnement (zowel tijdens als na de contra